# Mínimo, Máximo, e Mistura

Pense Bayes, Segunda Edição

Copyright 2020 Allen B. Downey

Licença: [Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/)

In [1]:
# If we're running on Colab, install empiricaldist
# https://pypi.org/project/empiricaldist/

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install empiricaldist

In [2]:
# Get utils.py

from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)
    
download('https://github.com/AllenDowney/ThinkBayes2/raw/master/soln/utils.py')

In [3]:
from utils import set_pyplot_params
set_pyplot_params()

No capítulo anterior, calculámos as distribuições de somas.

Neste capítulo, vamos calcular as distribuições de mínimos e máximos, e utilizá-las para resolver tanto os problemas a avançar como os inversos.

Depois veremos as distribuições que são misturas de outras distribuições, que se revelarão particularmente úteis para fazer previsões.

Mas vamos começar com uma ferramenta poderosa para trabalhar com distribuições, a função de distribuição cumulativa.

## Funções de Distribuição Cumulativa

Até agora temos utilizado funções de massa probabilística para representar distribuições.

Uma alternativa útil é a **função de distribuição cumulativa**, ou CDF.

Como exemplo, vou utilizar a distribuição posterior do problema do Euro, que calculámos em <<_BayesianEstimation>>>.

Aqui está o uniforme anterior com que começámos.

In [4]:
import numpy as np
from empiricaldist import Pmf

hypos = np.linspace(0, 1, 101)
pmf = Pmf(1, hypos)
data = 140, 250

E aqui está a actualização.

In [5]:
from scipy.stats import binom

def update_binomial(pmf, data):
    """Update pmf using the binomial distribution."""
    k, n = data
    xs = pmf.qs
    likelihood = binom.pmf(k, n, xs)
    pmf *= likelihood
    pmf.normalize()

In [6]:
update_binomial(pmf, data)

O CDF é a soma cumulativa do PMF, pelo que podemos calculá-lo desta forma:

In [7]:
cumulative = pmf.cumsum()

Aqui está o que parece, juntamente com o PMF.

In [8]:
from utils import decorate

def decorate_euro(title):
    decorate(xlabel='Proportion of heads (x)',
             ylabel='Probability',
             title=title)

In [9]:
cumulative.plot(label='CDF')
pmf.plot(label='PMF')
decorate_euro(title='Posterior distribution for the Euro problem')

O intervalo do CDF é sempre de 0 a 1, em contraste com o PMF, onde o máximo pode ser qualquer probabilidade.

O resultado do `cumsum` é um Pandas `Série`, pelo que podemos utilizar o operador de parênteses para seleccionar um elemento:

In [10]:
cumulative[0.61]

O resultado é de cerca de 0,96, o que significa que a probabilidade total de todas as quantidades inferiores ou iguais a 0,61 é de 96%.

Para ir pelo outro lado --- para procurar uma probabilidade e obter o quantil correspondente --- podemos usar a interpolação:

In [11]:
from scipy.interpolate import interp1d

ps = cumulative.values
qs = cumulative.index

interp = interp1d(ps, qs)
interp(0.96)

O resultado é de cerca de 0,61, o que confirma que o percentil 96 desta distribuição é de 0,61.

O `empiricaldist' fornece uma classe chamada `Cdf' que representa uma função de distribuição cumulativa.

Dado um `Pmf`, é possível calcular um `Cdf` como este:

In [12]:
cdf = pmf.make_cdf()

`make_cdf` utiliza `np.cumsum` para calcular a soma cumulativa das probabilidades.

Pode utilizar parênteses para seleccionar um elemento de um `Cdf`:

In [13]:
cdf[0.61]

Mas se procurar uma quantidade que não está na distribuição, recebe um "KeyError".



In [14]:
try:
    cdf[0.615]
except KeyError as e:
    print(repr(e))

Para evitar este problema, pode chamar um `Cdf` como função, utilizando parênteses.

Se o argumento não aparecer no `Cdf', interpola entre quantidades.

In [15]:
cdf(0.615)

Seguindo o outro caminho, pode utilizar o `quantile` para procurar uma probabilidade cumulativa e obter a quantidade correspondente:



In [16]:
cdf.quantile(0.9638303)

O "Cdf" também fornece "intervalo_credível", que calcula um intervalo credível que contém a probabilidade dada:



In [17]:
cdf.credible_interval(0.9)

Os CDF e os PMF são equivalentes no sentido em que contêm o

a mesma informação sobre a distribuição, e pode sempre converter

de um para o outro.

Dado um `Cdf`, pode obter o equivalente a `Pmf` como este:

In [18]:
pmf = cdf.make_pmf()

`make_pmf` utiliza `np.diff` para calcular as diferenças entre probabilidades cumulativas consecutivas.

Uma razão pela qual os objectos `Cdf` são úteis é que calculam quantis de forma eficiente.

Outra é que facilitam o cálculo da distribuição de um máximo ou mínimo, como veremos na secção seguinte.

## Os Três Melhores de Quatro

Em *Dungeons & Dragons*, cada personagem tem seis atributos: força, inteligência, sabedoria, destreza, constituição, e carisma.

Para gerar um novo personagem, os jogadores lançam quatro dados de 6 faces para cada atributo e somam os três melhores.

Por exemplo, se eu lançar por força e conseguir 1, 2, 3, 4 nos dados, a força do meu personagem seria a soma de 2, 3, e 4, que é 9.

Como um exercício, vamos descobrir a distribuição destes atributos.

Depois, para cada personagem, vamos descobrir a distribuição do seu melhor atributo.

Vou importar duas funções do capítulo anterior: "make_die", que faz um "Pmf" que representa o resultado de enrolar um dado, e "add_dist_seq", que pega numa sequência de objectos "Pmf" e calcula a distribuição da sua soma.

Aqui está um `Pmf` que representa um dado de seis lados e uma sequência com três referências a ele.

In [19]:
from utils import make_die

die = make_die(6)
dice = [die] * 3

E aqui está a distribuição da soma de três dados.

In [20]:
from utils import add_dist_seq

pmf_3d6 = add_dist_seq(dice)

Aqui está o que parece:

In [21]:
def decorate_dice(title=''):
    decorate(xlabel='Outcome',
             ylabel='PMF',
             title=title)

In [22]:
pmf_3d6.plot()
decorate_dice('Distribution of attributes')

Se lançarmos quatro dados e somarmos os três melhores, o cálculo da distribuição da soma é um pouco mais complicado.

Vou estimar a distribuição através da simulação de 10.000 rolos.

Primeiro vou criar um conjunto de valores aleatórios de 1 a 6, com 10.000 filas e 4 colunas:

In [23]:
n = 10000
a = np.random.randint(1, 7, size=(n, 4))

Para encontrar os três melhores resultados em cada fila, vou utilizar `sort` com `eixo=1`, que classifica as filas por ordem ascendente.

In [24]:
a.sort(axis=1)

Finalmente, vou seleccionar as três últimas colunas e adicioná-las.

In [25]:
t = a[:, 1:].sum(axis=1)

Agora `t` é um conjunto com uma única coluna e 10.000 filas.

Podemos calcular o PMF dos valores em `t` como este:

In [26]:
pmf_best3 = Pmf.from_seq(t)

A figura seguinte mostra a distribuição da soma de três dados, `pmf_3d6`, e a distribuição dos três melhores de quatro, `pmf_best3`.

In [27]:
pmf_3d6.plot(label='sum of 3 dice')
pmf_best3.plot(label='best 3 of 4', style='--')

decorate_dice('Distribution of attributes')

Como seria de esperar, a escolha dos três melhores de quatro tende a produzir valores mais elevados.

Em seguida, encontraremos a distribuição para o máximo de seis atributos, cada um a soma dos três melhores de quatro dados.

## Máximo

Para calcular a distribuição de um máximo ou mínimo, podemos fazer bom uso da função de distribuição cumulativa.

Primeiro, vou calcular o `Cdf` das três melhores de quatro distribuições:

In [28]:
cdf_best3 = pmf_best3.make_cdf()

Recordar que `Cdf(x)` é a soma das probabilidades para quantidades inferiores ou iguais a `x`.

Equivalentemente, é a probabilidade de um valor aleatório escolhido da distribuição ser inferior ou igual a `x`.

Agora suponhamos que retiro 6 valores desta distribuição.

A probabilidade de todos os 6 serem inferiores ou iguais a `x` é `Cdf(x)`elevada para a 6ª potência, que podemos calcular assim:

In [29]:
cdf_best3**6

Se todos os 6 valores forem inferiores ou iguais a `x', isso significa que o seu máximo é inferior ou igual a `x'.

Assim, o resultado é o CDF do seu máximo.

Podemos convertê-lo a um objecto `Cdf`, como este:

In [30]:
from empiricaldist import Cdf

cdf_max6 = Cdf(cdf_best3**6)

E calcular o equivalente `Pmf` como este:

In [31]:
pmf_max6 = cdf_max6.make_pmf()

A figura seguinte mostra o resultado.

In [32]:
pmf_max6.plot(label='max of 6 attributes')

decorate_dice('Distribution of attributes')

A maioria das personagens tem pelo menos um atributo superior a 12; quase 10% delas têm um 18.

A figura seguinte mostra os CDFs para as três distribuições que calculámos.

In [33]:
import matplotlib.pyplot as plt

cdf_3d6 = pmf_3d6.make_cdf()
cdf_3d6.plot(label='sum of 3 dice')

cdf_best3 = pmf_best3.make_cdf()
cdf_best3.plot(label='best 3 of 4 dice', style='--')

cdf_max6.plot(label='max of 6 attributes', style=':')

decorate_dice('Distribution of attributes')
plt.ylabel('CDF');

O `Cdf' fornece o `max_dist', que faz o mesmo cálculo, pelo que também podemos calcular o `Cdf' do máximo desta forma:

In [34]:
cdf_max_dist6 = cdf_best3.max_dist(6)

E podemos confirmar que as diferenças são pequenas.

In [35]:
np.allclose(cdf_max_dist6, cdf_max6)

Na secção seguinte, encontraremos a distribuição do mínimo.

O processo é semelhante, mas um pouco mais complicado.

Veja se consegue descobrir antes de continuar.

## Mínimo

Na secção anterior calculámos a distribuição do melhor atributo de um personagem.

Agora vamos calcular a distribuição do pior.

Para calcular a distribuição do mínimo, utilizaremos o **complementar CDF**, que podemos computar desta forma:

In [36]:
prob_gt = 1 - cdf_best3

Como o nome da variável sugere, o CDF complementar é a probabilidade de um valor da distribuição ser maior do que `x`.

Se retirarmos 6 valores da distribuição, a probabilidade de que todos os 6 excedam `x` é:

In [37]:
prob_gt6 = prob_gt**6

Se todos os 6 excederem `x', isso significa que o seu mínimo excede `x', então `prob_gt6' é o CDF complementar do mínimo.

E isso significa que podemos calcular o CDF do mínimo desta forma:

In [38]:
prob_le6 = 1 - prob_gt6

O resultado é uma "Série Pandas" que representa o CDF do mínimo de seis atributos.  Podemos colocar esses valores num objecto `Cdf` como este:

In [39]:
cdf_min6 = Cdf(prob_le6)

Aqui está o que parece, juntamente com a distribuição do máximo.

In [40]:
cdf_min6.plot(color='C4', label='minimum of 6')
cdf_max6.plot(color='C2', label='maximum of 6', style=':')
decorate_dice('Minimum and maximum of six attributes')
plt.ylabel('CDF');

O `Cdf' fornece o `min_dist', que faz o mesmo cálculo, pelo que também podemos calcular o `Cdf' do mínimo desta forma:

In [41]:
cdf_min_dist6 = cdf_best3.min_dist(6)

E podemos confirmar que as diferenças são pequenas.

In [42]:
np.allclose(cdf_min_dist6, cdf_min6)

Nos exercícios no final deste capítulo, utilizará distribuições do mínimo e do máximo para fazer inferências Bayesianas.

Mas primeiro vamos ver o que acontece quando misturamos as distribuições.

## Mistura

Nesta secção vou mostrar como podemos calcular uma distribuição que é uma mistura de outras distribuições.

Vou explicar o que isso significa com alguns exemplos simples;

então, mais proveitosamente, veremos como estas misturas são utilizadas para fazer previsões.

Aqui está outro exemplo inspirado em *Dungeons & Dragons*:

* Suponha que a sua personagem está armada com uma adaga numa mão e uma espada curta na outra.

* Durante cada ronda, ataca-se um monstro com uma das suas duas armas, escolhida ao acaso.

* A adaga causa um morto de 4 lados de dano; a espada curta causa um morto de 6 lados de dano.

Qual é a distribuição dos danos que inflige em cada ronda?

Para responder a esta pergunta, farei um `Pmf` para representar os dados de 4 e 6 faces:

In [43]:
d4 = make_die(4)
d6 = make_die(6)

Agora, vamos calcular a probabilidade de infligir 1 ponto de dano.

* Se atacou com a adaga, é 1/4.

* Se atacou com a espada curta, é 1/6.

Porque a probabilidade de escolher uma das armas é 1/2, a probabilidade total é a média:

In [44]:
prob_1 = (d4(1) + d6(1)) / 2
prob_1

Para os resultados 2, 3, e 4, a probabilidade é a mesma, mas para 5 e 6 é diferente, porque esses resultados são impossíveis com o dado de 4 lados.

In [45]:
prob_6 = (d4(6) + d6(6)) / 2
prob_6

Para calcular a distribuição da mistura, poderíamos percorrer os possíveis resultados e calcular as suas probabilidades.

Mas podemos fazer o mesmo cálculo utilizando o operador `+`:

In [46]:
mix1 = (d4 + d6) / 2

Eis como se parece a mistura destas distribuições.

In [47]:
mix1.bar(alpha=0.7)
decorate_dice('Mixture of one 4-sided and one 6-sided die')

Agora suponha que está a combater três monstros:

* Um tem um clube, que causa um dado de 4 lados de danos.

* Um tem um maça, o que causa um morto de 6 lados.

* e um tem um quarto de pessoal, o que também causa uma morte de 6 lados. 

Como o tumulto é desorganizado, é atacado por um destes monstros em cada ronda, escolhido ao acaso.

Para encontrar a distribuição dos danos que infligem, podemos calcular uma média ponderada das distribuições, como esta:

In [48]:
mix2 = (d4 + 2*d6) / 3

Esta distribuição é uma mistura de um dado de 4 faces e dois dados de 6 faces.

Aqui está o que parece.

In [49]:
mix2.bar(alpha=0.7)
decorate_dice('Mixture of one 4-sided and two 6-sided die')

Nesta secção utilizámos o operador `+`, que adiciona as probabilidades nas distribuições, não confundir com `Pmf.add_dist`, que calcula a distribuição da soma das distribuições.

Para demonstrar a diferença, vou utilizar `Pmf.add_dist` para calcular a distribuição do dano total feito por ronda, que é a soma das duas misturas:

In [50]:
total_damage = Pmf.add_dist(mix1, mix2)

E aqui está o que parece.

In [51]:
total_damage.bar(alpha=0.7)
decorate_dice('Total damage inflicted by both parties')

## Misturas gerais

Na secção anterior calculámos as misturas de uma forma *ad hoc*.

Agora vamos ver uma solução mais geral.

Em futuros capítulos, utilizaremos esta solução para gerar previsões para problemas do mundo real, e não apenas jogos de role-playing.

Mas se me suportarem, vamos continuar o exemplo anterior para mais uma secção.

Suponhamos que mais três monstros se juntam ao combate, cada um deles com um machado de batalha que causa um morto de 8 lados de dano.

Ainda assim, apenas um monstro ataca por ronda, escolhido ao acaso, pelo que os danos que infligem são uma mistura de:

* Um dado de 4 faces,

* Dois dados de 6 faces, e

* Três dados de 8 faces.

Vou utilizar um `Pmf` para representar um monstro escolhido aleatoriamente:

In [52]:
hypos = [4,6,8]
counts = [1,2,3]
pmf_dice = Pmf(counts, hypos)
pmf_dice.normalize()
pmf_dice

Esta distribuição representa o número de lados do coto que vamos enrolar e a probabilidade de enrolar cada um deles.

Por exemplo, um dos seis monstros tem uma adaga, por isso a probabilidade é de $1/6$ de enrolarmos um dado de 4 faces.

De seguida farei uma sequência de objectos "Pmf" para representar os dados:

In [53]:
dice = [make_die(sides) for sides in hypos]

Para calcular a distribuição da mistura, vou calcular a média ponderada dos dados, utilizando as probabilidades em `pmf_dice` como os pesos.

Para expressar este cálculo de forma concisa, é conveniente colocar as distribuições num Pandas "DataFrame":

In [54]:
import pandas as pd

pd.DataFrame(dice)

O resultado é um "DataFrame" com uma linha para cada distribuição e uma coluna para cada resultado possível.

Nem todas as filas têm o mesmo comprimento, por isso Pandas preenche os espaços extra com o valor especial "NaN", que significa "não um número".

Podemos utilizar `fillna` para substituir os valores `NaN` por 0.

O passo seguinte é multiplicar cada linha pelas probabilidades em `pmf_dice', o que se torna mais fácil se transpusermos a matriz de modo a que as distribuições corram pelas colunas em vez de pelas linhas:

In [55]:
df = pd.DataFrame(dice).fillna(0).transpose()

In [56]:
df

Agora podemos multiplicar pelas probabilidades em `pmf_dice`:



In [57]:
df *= pmf_dice.ps

In [58]:
df

E somar as distribuições ponderadas:

In [59]:
df.sum(axis=1)

O argumento `eixo=1` significa que queremos somar ao longo das filas.

O resultado é uma `Série Pandas`.

Juntando tudo, aqui está uma função que faz uma mistura ponderada de distribuições.

In [60]:
def make_mixture(pmf, pmf_seq):
    """Make a mixture of distributions."""
    df = pd.DataFrame(pmf_seq).fillna(0).transpose()
    df *= np.array(pmf)
    total = df.sum(axis=1)
    return Pmf(total)

O primeiro parâmetro é um `Pmf` que mapeia desde cada hipótese até uma probabilidade.

O segundo parâmetro é uma sequência de objectos `Pmf`, um para cada hipótese.

Podemos chamar-lhe assim:

In [61]:
mix = make_mixture(pmf_dice, dice)

E aqui está o que parece.

In [62]:
mix.bar(label='mixture', alpha=0.6)
decorate_dice('Distribution of damage with three different weapons')

Nesta secção utilizei Pandas para que a "mistura_make_mixture" fosse concisa, eficiente e, esperemos, não demasiado difícil de compreender.

Nos exercícios no final do capítulo, terá a oportunidade de praticar com misturas, e voltaremos a utilizar o `make_mixture` no próximo capítulo.

## Resumo

Este capítulo introduz o objecto `Cdf`, que representa a função de distribuição cumulativa (CDF).

Um `Pmf` e o correspondente `Cdf` são equivalentes no sentido em que contêm a mesma informação, pelo que se pode converter de um para o outro.  

A principal diferença entre elas é o desempenho: algumas operações são mais rápidas e fáceis com um `Pmf`; outras são mais rápidas com um `Cdf`.

Neste capítulo utilizámos objectos `Cdf` para calcular distribuições de máximos e mínimos; estas distribuições são úteis para inferência se nos for dado um máximo ou mínimo como dados.

Verá alguns exemplos nos exercícios, e em capítulos futuros.

Também calculámos misturas de distribuições, que utilizaremos no próximo capítulo para fazer previsões.

Mas primeiro talvez queira trabalhar nestes exercícios.

## Exercícios

**Exercício:** Quando se gera um carácter D&D, em vez de lançar dados, pode-se utilizar a "matriz padrão" de atributos, que é 15, 14, 13, 12, 10, e 8.

Acha que é melhor usar a matriz padrão ou (literalmente) lançar os dados?

Comparar a distribuição dos valores na matriz padrão com a distribuição que calculámos para os três melhores de quatro:

* Que distribuição tem uma média mais elevada?  Utilize o método "meio".

* Que distribuição tem um desvio padrão mais elevado?  Utilize o método `std`.

* O valor mais baixo na matriz padrão é 8. Para cada atributo, qual é a probabilidade de obter um valor inferior a 8? Se lançar os dados seis vezes, qual é a probabilidade de pelo menos um dos seus atributos ser inferior a 8?

* O valor mais alto na matriz padrão é 15.  Para cada atributo, qual é a probabilidade de obter um valor superior a 15?  Se lançar os dados seis vezes, qual é a probabilidade de que pelo menos um dos seus atributos seja maior do que 15?

Para começar, aqui está um `Cdf` que representa a distribuição de atributos na matriz padrão:

In [63]:
standard = [15,14,13,12,10,8]
cdf_standard = Cdf.from_seq(standard)

Podemos compará-lo com a distribuição de atributos que se obtém ao lançar quatro dados ao somar os três melhores.

In [64]:
cdf_best3.plot(label='best 3 of 4', color='C1', style='--')
cdf_standard.step(label='standard set', color='C7')

decorate_dice('Distribution of attributes')
plt.ylabel('CDF');

Tracei o `cdf_standard` como uma função de passo para mostrar mais claramente que contém apenas algumas quantidades.

In [65]:
# Solution goes here

In [66]:
# Solution goes here

In [67]:
# Solution goes here

In [68]:
# Solution goes here

In [69]:
# Solution goes here

In [70]:
# Solution goes here

**Exercício:** Suponha que está a combater três monstros:

* Um está armado com uma espada curta que causa um morto de 6 lados de danos,

* Um está armado com um machado de batalha que causa um morto de 8 lados de danos, e

* Um está armado com uma espada bastarda que causa um morto de 10 lados de danos.

Um dos monstros, escolhido ao acaso, ataca-o e faz 1 ponto de dano.

Que monstro pensa que foi?  Calcule a probabilidade posterior de cada monstro ter sido o atacante.

Se o mesmo monstro o atacar novamente, qual é a probabilidade de sofrer 6 pontos de dano?

Dica: Calcule uma distribuição posterior como já fizemos anteriormente e passe-a como um dos argumentos para `make_mixture'.

In [71]:
# Solution goes here

In [72]:
# Solution goes here

In [73]:
# Solution goes here

In [74]:
# Solution goes here

**Exercício: Henri Poincaré foi um matemático francês que ensinou na Sorbonne por volta de 1900. A seguinte anedota sobre ele é provavelmente ficção, mas constitui um problema de probabilidade interessante.

Supostamente PoincarÃ© suspeitava que a sua padaria local vendia pães mais leves do que o peso anunciado de 1 kg, por isso todos os dias durante um ano comprava um pão, levava-o para casa e pesava-o. No final do ano, traçou a distribuição das suas medidas e mostrou que se enquadrava numa distribuição normal com média de 950 g e desvio padrão de 50 g. Levou esta prova à polícia do pão, que deu um aviso ao padeiro.


Porquê? Porque a forma da nova distribuição era assimétrica. Ao contrário da distribuição normal, foi inclinada para a direita, o que é consistente com a hipótese de que o padeiro ainda estava a fazer pães de 950 g, mas dando deliberadamente a PoincarÃ© os mais pesados.

Para ver se esta anedota é plausível, suponhamos que quando o padeiro vê PoincarÃ© a chegar, ele rouba "não" pães e dá a PoincarÃ© o mais pesado.  Quantos pães o padeiro teria de levantar para fazer a média do máximo de 1000 g?

Para começar, irei gerar um ano de dados a partir de uma distribuição normal com os parâmetros indicados.

In [75]:
mean = 950
std = 50

np.random.seed(17)
sample = np.random.normal(mean, std, size=365)

In [76]:
# Solution goes here

In [77]:
# Solution goes here